In [2]:
import logging
logging.basicConfig(filename = '../logging/BrainNetsLog.log', 
                    filemode='w', 
                    format = '%(asctime)s %(name)s %(lineno)d %(funcName)s %(levelname)s:%(message)s', 
                    level = logging.INFO)
logger = logging.getLogger('Jupyter Notebook')

In [6]:
import BrainNets as bn
reload(bn)
import dataOperator as do
reload(do)
import models.sectorNet as sn
reload(sn)

import time
import theano
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline 

In [7]:
sectorNet = sn.SectorNet('/home/mountain/Desktop/BrainNets/code/configs/modelConfigs/SectorNetModel.cfg')
trainFunc = sectorNet.trainFunction()
valAndTestFunc = sectorNet.valAndTestFunction()

In [10]:
sectorNet.summary()

---------------------------------------------------------------------------

    Layer           Output shape         W shape             
1   Input           (None, 1, 17, 17, 17)                     
2   Conv3D          (None, 30, 15, 15, 15) (30, 1, 3, 3, 3)    
3   Concat          (None, 31, 15, 15, 15)                     
4   Conv3D          (None, 30, 13, 13, 13) (30, 31, 3, 3, 3)   
5   Concat          (None, 61, 13, 13, 13)                     
6   Conv3D          (None, 40, 11, 11, 11) (40, 61, 3, 3, 3)   
7   Concat          (None, 101, 11, 11, 11)                     
8   Conv3D          (None, 40, 9, 9, 9)  (40, 101, 3, 3, 3)  
9   Concat          (None, 141, 9, 9, 9)                     
10  Conv3D          (None, 40, 7, 7, 7)  (40, 141, 3, 3, 3)  
11  Concat          (None, 181, 7, 7, 7)                     
12  Conv3D          (None, 40, 5, 5, 5)  (40, 181, 3, 3, 3)  
13  Concat          (None, 221, 5, 5, 5)                     
14  Conv3D          (None, 50, 3, 3, 3)  

In [14]:
a = [1, 1, 1]

In [15]:
b = set(a)

In [16]:
b

{1}

In [17]:
len(b)

1

In [19]:
list(b)[0]

1

In [8]:
trainCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                           sourceDataDirectory = '../data/SimpleExploreSomeData/', 
                           subStructure = 'non-enhancing',
                           cubeSize = 17,
                           trainOrVal = 'train',
                           onTheFly = True)
valCubes = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', 
                         sourceDataDirectory = '../data/SimpleExploreSomeData/',
                         subStructure = 'non-enhancing',
                         cubeSize = 17,
                         onTheFly = True, 
                         trainOrVal = 'val')

In [9]:
numEpochs = 2

In [10]:
trainingLoss = []
validationLoss = []
validationACC = []

In [1]:
for epoch in range(numEpochs):
    logger.info('{} th of {} epochs'.format(epoch, numEpochs))
    
    trainError = 0
    startTime = time.time()
    trainBatch = 0
    
#     for batch in synthsisTrain.generateSynthesisData(1000, group0 = (0,5), group1 = (1, 5)):
    for batch in trainCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        trainLoss, trainACC = trainFunc(inputs, targets / 3)

        trainError += trainLoss
        trainBatch += 1
        
        print 'trainLoss: ', trainLoss, 'trainACC: ', trainACC

    valError = 0
    valACC = 0
    valBatches = 0
        
#     for batch in synthsisVal.generateSynthesisData(1000, group0 = (1,5), group1 = (1, 5)):
    for batch in valCubes.fetchCubes(batchSize = 100, parallel = True):
        inputs, targets = batch
        targets = targets
        valAndTestLoss, valAndTestACC = valAndTestFunc(inputs, targets / 3)
        valError += valAndTestLoss
        valACC += valAndTestACC
        valBatches += 1
        
    print "Epoch {} of {} took {:.3f}s".format(epoch + 1, numEpochs, time.time() - startTime)
    print "  training loss:\t\t{:.6f}".format(trainError / trainBatch)
    print "  validation loss:\t\t{:.6f}".format(valError / valBatches)
    print "  validation accuracy:\t\t{:.2f} %".format(valACC / valBatches * 100)
    
    trainingLoss.append(trainError / trainBatch)
    validationLoss.append(valError / valBatches)
    validationACC.append(valACC / valBatches * 100)

NameError: name 'numEpochs' is not defined

In [ ]:
plt.plot(trainingLoss)
plt.ylabel('trainingLoss')
plt.show()

In [ ]:
plt.plot(validationLoss)
plt.ylabel('validationLoss')
plt.show()

In [ ]:
plt.plot(validationACC)
plt.ylabel('validationACC')
plt.show()

In [ ]:
theano.printing.debugprint(trainFunc)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara1/',parallel = False)

In [ ]:
%time do.normalizeDataSet(dataPath = '../data/trainParallelNorm/', normalizedDataDir = '../data/outPutPara2/',parallel = True)

In [ ]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData1/',
             cubeSize = 33, 
             parallel = False)

In [ ]:
do.makeCubesOnTheFly(dataPath = '../data/SimpleExploreSomeData/', 
             cubeDirectory = '../data/tryGenerateCubeData/',
             cubeSize = 33, 
             parallel = True)

In [ ]:
synthsisTrain = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', batchSize = 1000)
synthsisVal = do.cubesGetor(cubeDirectory = '../data/tryGenerateCubeData/', trainOrVal = 'val', batchSize = 1000)

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = False):
    print time.time() - startTime
    startTime = time.time()

In [ ]:
startTime = time.time()
for inputs, targets in trainCubes.fetchCubes(batchSize = 100, parallel = True):
    print time.time() - startTime
    startTime = time.time()

In [1]:
import numpy as np

In [2]:
x = [1, 2, 3]

In [3]:
b = np.arange(10)

In [4]:
b

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
b[1]

1

In [6]:
x[b[1]]

2

In [7]:
b.dtype

dtype('int64')

In [15]:
c = np.asarray([1,2,3])

In [16]:
c

array([1, 2, 3])

In [17]:
c.shape

(3,)

In [18]:
c.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [19]:
d = np.arange(12).reshape((3, 4))

In [20]:
d

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [22]:
d - c[:, np.newaxis]

array([[-1,  0,  1,  2],
       [ 2,  3,  4,  5],
       [ 5,  6,  7,  8]])

In [23]:
lc = (1, 2, 3)

In [24]:
rc = (4, 5, 6)

In [26]:
f = zip(lc, rc)

In [27]:
f

[(1, 4), (2, 5), (3, 6)]

In [29]:
type(f)

list